In [1]:
# -*- coding: utf-8 -*-
import heapq
import random
import numpy as np 

In [2]:
# 假設的各推薦順序 ( 並無使用
recommend = [[1,2,3,4,5],   
             [1,3,5,2,4],
             [1,4,3,5,2],
             [2,5,3,1,4],
             [2,3,1,4,5],
             [2,1,3,4,5],
             [3,4,5,1,2],
             [3,5,4,2,1],
             [4,1,2,3,5],
             [5,4,3,2,1]]

In [3]:
pop_number = 10  # 總共有幾個個體
rounds = 10  # 目前沒有使用
recom_number = 50 # 每個個體產生多少個推薦食譜個數

In [55]:
next_generation = []
pop_storage = []
# recommend = []
weight = []

# variable - recommend : 各個母體所產生之推薦順序
# variable - pop_storage : 所有個體所在的 list
# variable - fitness : 儲存所有個體的 適應值[0] 和 其對應的個體[1] 
# variable - weight : 用於影響 輪盤選擇 的權重
# variable - next_generation : 子代


pop_storage = initial(pop_number) 

fitness = initial_fitness()
fitness = cal_fitness(fitness,recom_number)
fitness.sort(key=takeFirst,reverse=True)     # 以 fitness_value 做排序 [0]

keep(pop_storage, fitness, pop_number, 0.2, next_generation)
mutation(pop_storage, fitness, pop_number, 0.2, next_generation)

weight = weight_produce(pop_number)

# Random choose for crossover and mutation 
for i in range(1,4):   
    choose = random.choices(fitness, weights=weight ,k=int(pop_number*0.2))
    crossover(pop_storage[choose[0][1]],pop_storage[choose[1][1]],0.3,next_generation)
    if(i==3):
        muta_pop(pop_storage[choose[1][1]], 0, 0.1, next_generation)
    else:
        crossover(pop_storage[choose[1][1]],pop_storage[choose[0][1]],0.3,next_generation)
        
fitness = initial_fitness()
fitness = cal_fitness(fitness,recom_number)
fitness.sort(key=takeFirst,reverse=True)     # 以 fitness_value 做排序 [0]

[0.44, 0.44, 0.09, 0.03, 0.69, 0.31]
[0.37, 0.37, 0.22, 0.04, 0.63, 0.37]
[0.38, 0.38, 0.12, 0.12, 0.75, 0.25]
[0.34, 0.34, 0.19, 0.13, 0.97, 0.03]
[0.42, 0.42, 0.11, 0.05, 0.88, 0.12]
[0.31, 0.31, 0.21, 0.17, 0.85, 0.15]
[0.33, 0.33, 0.33, 0.01, 0.56, 0.44]
[0.4, 0.4, 0.14, 0.06, 0.99, 0.01]
[0.28, 0.28, 0.25, 0.19, 1.0, 0.0]
[0.34, 0.34, 0.25, 0.07, 0.85, 0.15]
['8', '3', '23', '41', '2', '1', '4', '6', '47', '8']
['1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
['4', '4', '4', '4', '4', '4', '4', '4', '4', '4']
['2', '2', '2', '2', '2', '2', '2', '2', '2', '2']
['4', '4', '4', '4', '4', '4', '4', '4', '4', '4']
['2', '2', '2', '2', '2', '2', '2', '2', '2', '2']
['4', '4', '4', '4', '4', '4', '4', '4', '4', '4']
['1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
['4', '4', '4', '4', '4', '4', '4', '4', '4', '4']
['3', '3', '3', '3', '3', '3', '3', '3', '3', '3']
['1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
['1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
['1', '1', '1', '1'

In [8]:
# 產生初始權重 = 母體

def initial(pop_number):
   
    pop_storage = []

    accept = 1


    while accept:

        check1 = 1
        check2 = 1

        while check1:
            frame = np.random.uniform(low=0, high=1, size=(4,))  # 亂數產生權重值
            first = []  # 前面四個權重
            for element in frame:
                element = round(element,2)
                first.append(element)


            first.sort(reverse=True)   
            # 僅滿足    W_s = W_k >= W_r >= W_t     ,   W_S + W_K + W_R + W_T = 1
            if(first[0] + first[1] + first[2] + first[3] == 1 and first[0] == first[1]):  
                check1 = 0

        while check2:

            frame = np.random.uniform(low=0, high=1, size=(1,))  # 亂數產生權重值
            second = []
            second.append(round(frame[0],2))
            second.append(round(1-second[0],2))           # W_pre + W_pop = 1
            second.sort(reverse=True)
            if(second[0] > second[1]):   # W_pre > W_pop
                check2 = 0

        pop = first + second
        if(check1 == 0 and check2 == 0):     # 通過所有條件才 append 
            pop_storage.append(pop)

        if(len(pop_storage) == pop_number):   
            accept = 0

    for i in pop_storage:
        print(i)
    
    return pop_storage

In [9]:
def initial_fitness():
    fitness = []
    for i in range(0,10):
        temp = [0,i]
        fitness.append(temp)
        
    return fitness

In [2]:
# 需有測試資料
# 計算 適應值 

import requests
import lxml
from bs4 import BeautifulSoup
import pymysql

# 打开数据库连接
db =  pymysql.connect(host='120.107.172.110',
                       user='jason',
                       passwd='2u4n0 au/6',
                       db='program',
                       charset='utf8')
cursor = db.cursor()

# input - pop : 對個體值計算適應值 (要根據資料庫的資料做計算
# output - fitness : 回傳個體適應值 二維陣列 [fitness_value][which_pop]



In [3]:
# 單純查看 sequence 用的
sql = 'SELECT recommend_seq FROM share'
cursor.execute(sql)
result = cursor.fetchall()
for i in result:
    x = list(i)
    target = str(x[0])
    t = target.split(',')
    print(t)
    for i in range(0,len(t)):
        print(t[i])

['15', '15', '15', '15', '15', '15', '15', '15', '15', '15']
15
15
15
15
15
15
15
15
15
15
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0']
0
0
0
0
0
0
0
0
0
0
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0']
0
0
0
0
0
0
0
0
0
0
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0']
0
0
0
0
0
0
0
0
0
0
['21', '21', '23', '21', '20', '21', '21', '20', '20', '21']
21
21
23
21
20
21
21
20
20
21
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0']
0
0
0
0
0
0
0
0
0
0
['31', '31', '32', '31', '30', '31', '31', '31', '31', '30']
31
31
32
31
30
31
31
31
31
30
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0']
0
0
0
0
0
0
0
0
0
0
['4', '4', '4', '4', '4', '4', '4', '4', '4', '4']
4
4
4
4
4
4
4
4
4
4
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0']
0
0
0
0
0
0
0
0
0
0
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0']
0
0
0
0
0
0
0
0
0
0
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0']
0
0
0
0
0
0
0
0
0
0
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0']
0
0
0
0
0
0
0
0
0
0
['17', '17', '17'

25
25
25
25
25
25
25
25
25
['1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
1
1
1
1
1
1
1
1
1
1
['37', '37', '37', '37', '36', '37', '37', '37', '37', '36']
37
37
37
37
36
37
37
37
37
36
['20', '20', '19', '20', '21', '20', '20', '21', '21', '20']
20
20
19
20
21
20
20
21
21
20
['49', '49', '49', '49', '49', '49', '49', '49', '49', '49']
49
49
49
49
49
49
49
49
49
49
['34', '34', '34', '34', '33', '34', '34', '34', '33', '34']
34
34
34
34
33
34
34
34
33
34
['36', '36', '36', '36', '36', '36', '36', '36', '36', '36']
36
36
36
36
36
36
36
36
36
36
['21', '21', '23', '21', '20', '21', '21', '20', '20', '21']
21
21
23
21
20
21
21
20
20
21
['19', '19', '19', '19', '19', '19', '19', '19', '19', '19']
19
19
19
19
19
19
19
19
19
19
['6', '6', '6', '6', '6', '6', '6', '6', '6', '6']
6
6
6
6
6
6
6
6
6
6
['9', '9', '9', '9', '9', '9', '9', '9', '9', '9']
9
9
9
9
9
9
9
9
9
9
['26', '26', '26', '26', '26', '26', '26', '26', '26', '26']
26
26
26
26
26
26
26
26
26
26
['42', '42', '42', '42', '41', '

In [26]:
# 單純測驗
sql = 'SELECT recipe_id FROM collect'
cursor.execute(sql)
result = cursor.fetchall()

for x in result:
    # print(x[0])
    sql2='select id from receipe where id= %s'
    cursor.execute(sql2,(str(x[0])))
    result2=cursor.fetchall()
    print(result2 , x)

((61280,),) (61280,)
((52613,),) (52613,)
((46641,),) (46641,)
((13811,),) (13811,)
((46103,),) (46103,)
((6521,),) (6521,)
((72041,),) (72041,)
((49965,),) (49965,)
((6085,),) (6085,)
((58918,),) (58918,)
((41387,),) (41387,)
((62544,),) (62544,)
((20702,),) (20702,)
((63902,),) (63902,)
((11115,),) (11115,)
((49429,),) (49429,)
((49327,),) (49327,)
((21373,),) (21373,)
((20377,),) (20377,)
() (42534,)
((15620,),) (15620,)
((50095,),) (50095,)
((33123,),) (33123,)
() (36702,)
((8094,),) (8094,)
() (37444,)
((26977,),) (26977,)
((11043,),) (11043,)
((20341,),) (20341,)
((52696,),) (52696,)
((16180,),) (16180,)
((75,),) (75,)
((51267,),) (51267,)
((12267,),) (12267,)
((67642,),) (67642,)
((41232,),) (41232,)
((59534,),) (59534,)
((4081,),) (4081,)
((59081,),) (59081,)
((12738,),) (12738,)
() (34428,)
((16180,),) (16180,)
((12932,),) (12932,)
((55614,),) (55614,)
((30938,),) (30938,)
((7147,),) (7147,)
((23998,),) (23998,)
((30874,),) (30874,)
((48825,),) (48825,)
((31923,),) (31923,)
()

In [38]:
def cal_fitness(fitness,recom_number):
                
    fitness = cal_share(fitness,recom_number)
    fitness = cal_collect(fitness,recom_number)
    fitness = cal_rate(fitness,recom_number)
    fitness = cal_watch(fitness,recom_number)
        
    return fitness

In [54]:
def cal_share(fitness,recom_number):
    
    count = [0,0,0,0,0,0,0,0,0,0]
    
    sql = 'SELECT recommend_seq FROM share'
    cursor.execute(sql)
    result = cursor.fetchall()
    for seq in result:
        temp = list(seq)
        s = str(temp[0])
        s = s.split(',')
        print(s)
        for i in range(0,len(s)):
            if( int(s[i]) != 0 ):
                fitness[i][0] = fitness[i][0] + 1/int(s[i])
                count[i] = count[i] + 1
                
                
    for i in range(0,len(fitness)):
        fitness[i][0] = fitness[i][0]*0.4 * (count[i]/recom_number)
        
    return fitness

In [50]:
def cal_collect(fitness,recom_number):
    count = [0,0,0,0,0,0,0,0,0,0]
    
    sql = 'SELECT recommend_seq FROM collect'
    cursor.execute(sql)
    result = cursor.fetchall()
    for seq in result:
        temp = list(seq)
        s = str(temp[0])
        s = s.split(',')
        for i in range(0,len(s)):
            if( int(s[i]) != 0 ):
                fitness[i][0] = fitness[i][0] + 1/int(s[i])
                count[i] = count[i] + 1
    
    for i in range(0,len(fitness)):
        fitness[i][0] = fitness[i][0]*0.3 * (count[i]/recom_number)
    
    return fitness

In [51]:
def cal_rate(fitness,recom_number):
    count = [0,0,0,0,0,0,0,0,0,0]
    
    sql = 'SELECT recommend_seq FROM share'
    cursor.execute(sql)
    result = cursor.fetchall()
    for seq in result:
        temp = list(seq)
        s = str(temp[0])
        s = s.split(',')
        for i in range(0,len(s)):
            if( int(s[i]) != 0 ):
                fitness[i][0] = fitness[i][0] + 1/int(s[i])
                count[i] = count[i] + 1
    
    for i in range(0,len(fitness)):
        fitness[i][0] = fitness[i][0]*0.2 * (count[i]/recom_number)
    
    return fitness

In [52]:
def cal_watch(fitness,recom_number):
    count = [0,0,0,0,0,0,0,0,0,0]
    
    sql = 'SELECT recommend_seq FROM share'
    cursor.execute(sql)
    result = cursor.fetchall()
    for seq in result:
        temp = list(seq)
        s = str(temp[0])
        s = s.split(',')
        for i in range(0,len(s)):
            if( int(s[i]) != 0 ):
                fitness[i][0] = fitness[i][0] + 1/int(s[i])
                count[i] = count[i] + 1
    
    for i in range(0,len(fitness)):
        fitness[i][0] = fitness[i][0]*0.1 * (count[i]/recom_number)
    
    return fitness

In [17]:
def takeFirst(lst):
    return lst[0]

In [18]:
# 保留
# argument - pop_storage 儲存各權重的 list
# argument - part 取多少部分保留

def keep(pop_storage, fitness, pop_number, part, next_generation):
    
    for i in range(0,int(pop_number*part)):
        next_generation.append(pop_storage[fitness[i][1]])
    

In [19]:
# 突變
# argument - part 取多少比率保留


def mutation(pop_storage, fitness, pop_number, part, next_generation):
    
    for j in range(0,int(pop_number*part)):
        
        # 利用高斯函數產生突變的變數
        
        mu, sigma = 0, 0.1       
        noise = np.random.normal(mu, sigma, [6])  
        
        
        #for i in range(0,len(noise)):
           # noise[i] = round(noise[i],2)   # 取小數點後兩位
        
        
        afterMutation = noise + pop_storage[fitness[j][1]]
        afterMutation = afterMutation.tolist()     # 將 np.array 轉成 list
        
        for i in range(0,len(afterMutation)):      # 取小數點後兩位
            afterMutation[i] = round(afterMutation[i],2)

        next_generation.append(afterMutation)

# 針對單一染色體進行突變
def muta_pop(pop, mu, sigma, next_generation):
    
    noise = np.random.normal(mu, sigma, [6])  
    for i in range(0,len(noise)):
        noise[i] = round(noise[i],2)   # 取小數點後兩位
    child = pop + noise
    child = child.tolist()
    
    for i in range(0,len(child)):
        child[i] = round(child[i],2)
    next_generation.append(child)
    

In [20]:
# 交配
# argument - pop1.pop2 兩個目標染色體
def crossover(pop1, pop2, alpha, next_generation):
    
    child = []
    
    for i in range(0,len(pop1)):
        gene = alpha*pop1[i] + (1-alpha)*pop2[i]
        child.append(gene)
    
    for i in range(0,len(child)):
        child[i] = round(child[i],2)
    
    next_generation.append(child)
    

In [21]:
# 產生 russia roulette 的權重
def weight_produce(pop_number):
    
    weight = []
    total = 0
    for i in range(1,pop_number+1):
        total = i + total
    
    for i in range(pop_number,0,-1):
        weight.append(round(i/total,2))
    
    return weight

In [14]:
# ------------------------------以下為測試區----------------------------------

In [ ]:

    
    check1 = 1
    check2 = 1
    
    while check1:
        frame = np.random.uniform(low=0, high=1, size=(4,))  # 亂數產生權重值
        first = []  # 前面四個權重
        for element in frame:
            element = round(element,2)
            first.append(element)
        
        
        first.sort(reverse=True)   
        # 僅滿足    W_s = W_k >= W_r >= W_t     ,   W_S + W_K + W_R + W_T = 1
        if(first[0] + first[1] + first[2] + first[3] == 1 and first[0] == first[1]):  
            print(first)
            check1 == 0
        
        
    while check2:
            
        frame = np.random.uniform(low=0, high=1, size=(1,))  # 亂數產生權重值
        second = []
        second.append(round(frame[0]),2)
        second.append(round(1-frame[0],2))           # W_pre + W_pop = 1
        second.sort(reverse=True)
        if(second[0] > second[1]):   # W_pre > W_pop
            check2 == 0
        
    pop = first + second
    if(check1 == 0 and check2 == 0):     # 通過所有條件才 append 
        pop_storage.append(pop)


In [28]:
check = 1
while check:
    print(check)

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


KeyboardInterrupt: 

In [ ]:
weight = weight_produce(10)

In [ ]:
a = [1.5423,-12.01512]
for i in range(0,len(a)):
    a[i] = round(a[i],2)
print(a)

In [26]:
number = 10
partt = 0.5
number = int(number*partt)
for i in range(0,number):
    print("let go")

let go
let go
let go
let go
let go


In [11]:
test = [[10,1],[9,2],[5,5]]
choose = random.choices(test, weights=[0.1,0.1,0.8] ,k=2)
print(choose)
print(test[1][1])

[[5, 5], [5, 5]]
2


In [17]:
# Mutation 


mu, sigma = 0, 0.1 
# creating a noise with the same dimension as the dataset (2,2) 
noise = np.random.normal(mu, sigma, [6]) 
test = [1,2,3,4,5,6]


for i in range(0,len(noise)):
    noise[i] = round(noise[i],2)


print(noise)

[-0.1  -0.06 -0.05  0.05 -0.1  -0.05]


In [16]:

nums = [1, 8, 2, 23, 7, -4, 18, 23, 24, 37, 2]

# 最大的3个数的索引
max_num_index_list = map(nums.index, heapq.nlargest(3, nums))

# 最小的3个数的索引
min_num_index_list = map(nums.index, heapq.nsmallest(3, nums))

print(list(max_num_index_list))
print(list(min_num_index_list))

[9, 8, 3]
[5, 0, 2]


In [11]:
aaa = [[45.73,0],[40.5,1],[97.43,2]]

aaa.sort(key=takeFirst,reverse=True)     # 以 fitness_value 做排序 [0]
print(aaa)
print(aaa[0][1])

choose = random.choices(aaa,k=2)
print(choose[0][1])
print(choose[1][1])

[[97.43, 2], [45.73, 0], [40.5, 1]]
2
0
1
